In [1]:
import pandas as pd
import numpy as np
import py_pcha
from py_pcha import PCHA
import matplotlib.pyplot as plt
from numpy.matlib import repmat

In [ ]:
dimensions = 15
examples = 100
X = np.random.random((dimensions, examples))

In [ ]:
X.shape

In [ ]:
import pixiedust

In [ ]:
from pip._internal import main
main(["install", "msgpack"])
main(["install", "pixiedust"])

In [ ]:
pip install --upgrade ipython

In [2]:
data=pd.read_csv("/Users/markomiholjcic/Documents/GitHub/ArchetypalAnalysis/data/us_states_covid19_daily.csv")
data

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201206,AK,35720.0,NaN,1042056.0,NaN,totalTestsViral,1077776.0,164.0,799.0,...,1077776,0,0,7b1d31e2756687bb9259b29195f1db6cdb321ea6,0,0,0,0,0,NaN
1,20201206,AL,269877.0,45962.0,1421126.0,NaN,totalTestsPeopleViral,1645041.0,1927.0,26331.0,...,1691003,12,0,19454ed8fe28fc0a7948fc0771b2f3c846c1c92e,0,0,0,0,0,NaN
2,20201206,AR,170924.0,22753.0,1614979.0,NaN,totalTestsViral,1763150.0,1076.0,9401.0,...,1785903,40,21,25fc83bffff5b32ba1a737be8e087fad9f4fde33,0,0,0,0,0,NaN
3,20201206,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,8c39eec317586b0c34fc2903e6a3891ecb00469e,0,0,0,0,0,NaN
4,20201206,AZ,364276.0,12590.0,2018813.0,NaN,totalTestsPeopleViral,2370499.0,2977.0,28248.0,...,2383089,25,242,7cf59da9e4bc31d905e179211313d08879880a85,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15628,20200124,WA,0.0,NaN,0.0,NaN,totalTestEncountersViral,0.0,NaN,NaN,...,0,0,0,82726df68eb97c98a4a6ee792349e547023147d5,0,0,0,0,0,NaN
15629,20200123,MA,NaN,NaN,NaN,NaN,totalTestsViral,2.0,NaN,NaN,...,0,0,0,76bc987d054b119a4e05a4a43742249c0b0568b6,0,0,0,0,0,NaN
15630,20200123,WA,0.0,NaN,0.0,NaN,totalTestEncountersViral,0.0,NaN,NaN,...,0,0,0,1c2229c239ffad5e5fdd9d76c641dc9166caf6ba,0,0,0,0,0,NaN
15631,20200122,MA,NaN,NaN,NaN,NaN,totalTestsViral,1.0,NaN,NaN,...,0,0,0,01f5dcd6631859503ef1b62d81d49e41d12fc1bd,0,0,0,0,0,NaN


In [ ]:
data=pd.read_csv("/Users/u0984549/Documents/GitHub/ArchetypalAnalysis/data/us_states_covid19_daily.csv")
data

Data for 50 states, DC, and 5 territories on December 6, 2020. Select states and territories with missing values were removed. 

In [3]:
dec6 = data.loc[data['date'] == 20201206]
dec6 = dec6[['state','positive','negative','totalTestResults','recovered', 'death']]
dec6 = dec6.dropna()
dec6_state = dec6[['state']]

dec6raw = dec6

dec6 = dec6[['positive','negative','totalTestResults','recovered', 'death']]
dec6 = dec6.T
#dec6_state
dec6_variables = pd.DataFrame([dec6.index]).T
dec6raw

,state,positive,negative,totalTestResults,recovered,death
0,AK,35720.0,1042056.0,1077776.0,7165.0,143.0
1,AL,269877.0,1421126.0,1645041.0,168387.0,3889.0
2,AR,170924.0,1614979.0,1763150.0,149490.0,2660.0
4,AZ,364276.0,2018813.0,2370499.0,56382.0,6950.0
6,CO,260581.0,1608829.0,3478160.0,13941.0,2724.0
7,CT,127715.0,3294383.0,3422098.0,9800.0,5146.0
8,DC,23136.0,711497.0,734633.0,16665.0,697.0
9,DE,39912.0,400854.0,778298.0,18851.0,793.0
12,GU,7004.0,79571.0,86575.0,6056.0,113.0
13,HI,18842.0,295153.0,701776.0,11958.0,262.0


In [5]:
dec6raw.to_csv('/Users/markomiholjcic/Documents/GitHub/ArchetypalAnalysis/data/COVID19.csv', index = False)

In [ ]:
dec6b = data.loc[data['date'] == 20201206]
dec6b = dec6b[['positive','negative','totalTestResults','recovered', 'death', 
              'probableCases','hospitalizedCurrently',
             'negativeRegularScore','positiveScore','deathIncrease',
              'totalTestsAntibody','dataQualityGrade', 'deathProbable']]
dec6b = dec6b.dropna()
dec6b = dec6b[:4]
dec6b.T

In [ ]:
dec6.to_csv('/Users/markomiholjcic/Documents/GitHub/ArchetypalAnalysis/data/COVID19.csv', index = False)

In [ ]:
dec6b.to_csv('/Users/markomiholjcic/Documents/GitHub/ArchetypalAnalysis/data/COVID19BIG.csv', index = False)

In [ ]:
dec6.to_csv('/Users/u0984549/Documents/GitHub/ArchetypalAnalysis/data/COVID19.csv', index = False)

In [ ]:
U = np.array([1,2,3])
X = np.array([[2,3,4,5], [5,6,7,8], [1,4,5,6]])

print(X)
print(X.T)

#(X[:, U] * X[:, U])

In [ ]:
np.dot(X, X.T)

In [ ]:
repmat(np.mat(np.diag(X)).T, 1, 5)

In [ ]:
repmat(np.mat(np.diag(X)), 5, 1)

In [ ]:
int(np.ceil(5 * np.random.rand()))

In [ ]:
len(range(0,5))

In [ ]:
np.array(range(5))

In [ ]:
t = np.diag(X)
print(t)
np.where(t != 2)[0]

In [ ]:
repmat(np.diag(X), 5, 1)

In [ ]:
X[:,U]*X[:,U]

In [ ]:
ind_t = 2
ind_t[0]

In [ ]:
X

In [ ]:
X[[ind_t],:] * X[ind_t]

In [ ]:
np(X + X[ind_t])

In [ ]:
np.lib.scimath.sqrt(X + X[ind_t])

In [ ]:
Kt = np.array([[1,2], [5,6], [8,9]])
Kt = pd.DataFrame(Kt)
Kt.to_csv('/Users/markomiholjcic/Documents/GitHub/ArchetypalAnalysis/data/small.csv', index = False)

In [ ]:
e = np.ones((4, 1))
e

In [ ]:
CtXtXC = [2,3]
S = [3,4]
XCtX = [1,1]
SST = 5
J = 5
g = (np.dot(CtXtXC, S) - XCtX) / (SST / J)
g

In [ ]:
a = np.arange(10)
a.clip(min=5)

In [ ]:
a = range(2)
print(a)

In [ ]:
X = np.matrix([[1, 2, 3],
    [4, 5, 6], [7,8,9]])
U = [0,1,2]
print(U)
print(X)
print(X[:, U])

In [ ]:
np.sum(np.multiply(X[:, U],X[:, U]))

In [ ]:
[int(np.ceil(len(U) * np.random.rand()))]

In [ ]:
noc = 3
I = [1,2]
i = [3]
j = 3

In [ ]:
len(I)

In [ ]:
Kt = np.array([[1,2], [5,6], [8,9]])
print(Kt)
i = [1]
[I,J] = Kt.shape
print(I)
print(J)

In [ ]:
index = np.array(range(J))
index[i] = -1
index

ind_t = i
sum_dist = np.zeros((1, J), np.complex128)
sum_dist

In [ ]:
np.random.random((noc, len(U)))

In [ ]:
result = Kt**2
result

In [ ]:
Kt2 = np.sum(Kt**2, axis=0)
print(Kt2)


In [ ]:
Kq = np.dot(Kt[:, i[0]], Kt)
Kq

In [ ]:
np.lib.scimath.sqrt(Kt2 - 2 * Kq + Kt2[i[0]])

In [ ]:
sum_dist -= np.lib.scimath.sqrt(Kt2 - 2 * Kq + Kt2[i[0]])
sum_dist

In [ ]:
index[i[0]] = i[0]
index

In [ ]:
i = i[1:]
i

In [ ]:
index

In [ ]:
t = np.where(index != 3)[0]
t

In [ ]:
Kt[:, ind_t].T

In [ ]:
Kq = np.dot(Kt[:, ind_t].T, Kt)
Kq

In [ ]:
np.dot(np.ones((noc, 1)), np.mat(np.sum(S, axis=0)))

In [ ]:
np.ones((noc, 1))

In [ ]:
np.sum(S, axis=0)

In [ ]:
np.mat(np.sum(S, axis=0))

In [ ]:
ind

In [ ]:
S

In [ ]:
np.sum(S, axis=0)

In [ ]:
np.sum(S, axis=1)

In [ ]:
l = [1,9,3,4,5,6]
max(zip(range(len(l)),l), key = lambda x:x[1])

In [ ]:
sum_dist

In [ ]:
index

In [ ]:
t = np.where(index != -1)[0]
t

In [ ]:
sum_dist = np.zeros((1,5), np.complex128)
sum_dist[0][0] += 2
sum_dist[0][4] += 5

print(sum_dist)
sum_dist[:,[2,3,4]][0].real


In [ ]:
index = np.array([0,2,-1,4,5])

In [ ]:
i = 2
i

In [ ]:
t = np.where(index != -1)[0]

In [ ]:
sum_dist

In [ ]:
max(sum_dist[:,[0,1,3,4]][0].real)

In [ ]:
vals

In [ ]:
print(pd.__version__)

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
j = [0,1,2]

In [ ]:
i = [1,2,0]

In [ ]:
csr_matrix((np.ones(len(i)), (i, j)), shape=(5, 3)).todense()

In [ ]:
U

In [ ]:
np.log(np.random.random((3, len(U))))

In [ ]:
X = np.ones((3, 5))*2
S = np.ones((3, 5))*8
g = S*X
g

In [ ]:
np.sum(XCtX.A * S.A)

In [ ]:
e = np.ones((5, 1))
e is array

In [ ]:
r = np.sum(g * g, axis=0)
r.shape

In [ ]:
e * np.sum(g * g, axis=0)

In [ ]:
#dec6_wo_st = dec6[['positive','negative','totalTestResults','recovered', 'death']]
dec6_wo_st = dec6.to_numpy()
dec6_wo_st

In [ ]:
XC, S, C, SSE, varexpl = PCHA(dec6_wo_st, noc=3, I = None, delta=0.1)

In [ ]:
XC

In [ ]:
np.random.random((3,5))

In [ ]:
np.ones((3,5))-0.5

In [ ]:
def furthest_sum(K, noc, i, exclude=[]):
    
    def max_ind_val(l):
        return max(zip(range(len(l)), l), key=lambda x: x[1])
    
    I, J = K.shape
    #print(I,J)
    index = np.array(range(J))
    index[exclude] = 0
    index[i] = -1
    ind_t = i
    sum_dist = np.zeros((1, J), np.complex128)

    if J > noc * I:
        Kt = K
        Kt2 = np.sum(Kt**2, axis=0)
        for k in range(1, noc + 11):
            if k > noc - 1:
                Kq = np.dot(Kt[:, i[0]], Kt)
                #Kt_shape = Kt.shape
                #Kts = Kt[:, i[0]]
                #Ktsp = Kts.shape
                #Kq_shape = Kq.shape
                #print(Kt_shape,Ktsp,Kq_shape)
                sum_dist -= np.lib.scimath.sqrt(Kt2 - 2 * Kq + Kt2[i[0]])
                index[i[0]] = i[0]
                i = i[1:]
            t = np.where(index != -1)[0]
            Kq = np.dot(Kt[:, ind_t].T, Kt)
            sum_dist += np.lib.scimath.sqrt(Kt2 - 2 * Kq + Kt2[ind_t])
            #print(Kt2)
            ind, val = max_ind_val(sum_dist[:, t][0].real)
            ind_t = t[ind]
            i.append(ind_t)
            index[ind_t] = -1
    else:
        if I != J or np.sum(K - K.T) != 0:  # Generate kernel if K not one
            Kt = K
            K = np.dot(Kt.T, Kt)
            K = np.lib.scimath.sqrt(
                repmat(np.diag(K), J, 1) - 2 * K + \
                repmat(np.mat(np.diag(K)).T, 1, J)
            )

        Kt2 = np.diag(K)  # Horizontal
        for k in range(1, noc + 11):
            if k > noc - 1:
                sum_dist -= np.lib.scimath.sqrt(Kt2 - 2 * K[i[0], :] + Kt2[i[0]])
                index[i[0]] = i[0]
                i = i[1:]
            t = np.where(index != -1)[0]
            sum_dist += np.lib.scimath.sqrt(Kt2 - 2 * K[ind_t, :] + Kt2[ind_t])
            ind, val = max_ind_val(sum_dist[:, t][0].real)
            ind_t = t[ind]
            i.append(ind_t)
            index[ind_t] = -1

    return i


In [ ]:
"""Principal Convex Hull Analysis (PCHA) / Archetypal Analysis."""

import numpy as np
from scipy.sparse import csr_matrix
from datetime import datetime as dt
import time

#from py_pcha.furthest_sum import furthest_sum


def PCHAm(X, noc, I=None, U=None, delta=0, verbose=False, conv_crit=1E-6, maxiter=500):
    """Return archetypes of dataset.
    Note: Commonly data is formatted to have shape (examples, dimensions).
    This function takes input and returns output of the transposed shape,
    (dimensions, examples).
    Parameters
    ----------
    X : numpy.2darray
        Data matrix in which to find archetypes
    noc : int
        Number of archetypes to find
    I : 1d-array
        Entries of X to use for dictionary in C (optional)
    U : 1d-array
        Entries of X to model in S (optional)
    Output
    ------
    XC : numpy.2darray
        I x noc feature matrix (i.e. XC=X[:,I]*C forming the archetypes)
    S : numpy.2darray
        noc x length(U) matrix, S>=0 |S_j|_1=1
    C : numpy.2darray
        noc x length(U) matrix, S>=0 |S_j|_1=1
    SSE : float
        Sum of Squared Errors
    varexlp : float
        Percent variation explained by the model
    """
    def S_update(S, XCtX, CtXtXC, muS, SST, SSE, niter):
        """Update S for one iteration of the algorithm."""
        noc, J = S.shape
        e = np.ones((noc, 1))
        #print(e.ndim)
        for k in range(niter):
            SSE_old = SSE
            #print(SSE_old)
            g = (np.dot(CtXtXC, S) - XCtX) / (SST / J)
            #print(g)
            g = g - e * np.sum(g.A * S.A, axis=0)
            #print(g)

            S_old = S
            #print(S_old)
            while True:
                S = (S_old - g * muS).clip(min=0)
                S = S / np.dot(e, np.sum(S, axis=0))
                #print(S)
                SSt = S * S.T
                #print(SSt)
                SSE = SST - 2 * np.sum(XCtX.A * S.A) + np.sum(CtXtXC.A * SSt.A)
                #print(SSE)
                #print(SSE_old)
                if SSE <= SSE_old * (1 + 1e-9):
                    muS = muS * 1.2
                    #print(muS)
                    break
                else:
                    muS = muS / 2

        return S, SSE, muS, SSt

    def C_update(X, XSt, XC, SSt, C, delta, muC, mualpha, SST, SSE, niter=1):
        """Update C for one iteration of the algorithm."""
        J, nos = C.shape

        if delta != 0:
            alphaC = np.sum(C, axis=0).A[0]
            C = np.dot(C, np.diag(1 / alphaC))

        #print(C)
        e = np.ones((J, 1))
        XtXSt = np.dot(X.T, XSt)
        #print(XtXSt)

        for k in range(niter):

            # Update C
            SSE_old = SSE
            #print(SSE_old)
            g = (np.dot(X.T, np.dot(XC, SSt)) - XtXSt) / SST
            #print(g)

            if delta != 0:
                g = np.dot(g, np.diag(alphaC))
            g = g.A - e * np.sum(g.A * C.A, axis=0)
            #print(g)
            
            C_old = C
            while True:
                C = (C_old - muC * g).clip(min=0)
                #print(C)
                nC = np.sum(C, axis=0) + np.finfo(float).eps
                C = np.dot(C, np.diag(1 / nC.A[0]))
                #print(C)
                
                if delta != 0:
                    Ct = C * np.diag(alphaC)
                else:
                    Ct = C

                XC = np.dot(X, Ct)
                CtXtXC = np.dot(XC.T, XC)
                #print(CtXtXC)
                SSE = SST - 2 * np.sum(XC.A * XSt.A) + np.sum(CtXtXC.A * SSt.A)
                #print(SSE)
                
                if SSE <= SSE_old * (1 + 1e-9):
                    muC = muC * 1.2
                    break
                else:
                    muC = muC / 2

            # Update alphaC
            SSE_old = SSE
            if delta != 0:
                g = (np.diag(CtXtXC * SSt).T / alphaC - np.sum(C.A * XtXSt.A)) / (SST * J)
                #print(g)
                alphaC_old = alphaC
                while True:
                    alphaC = alphaC_old - mualpha * g
                    alphaC[alphaC < 1 - delta] = 1 - delta
                    alphaC[alphaC > 1 + delta] = 1 + delta

                    XCt = np.dot(XC, np.diag(alphaC / alphaC_old))
                    CtXtXC = np.dot(XCt.T, XCt)
                    SSE = SST - 2 * np.sum(XCt.A * XSt.A) + np.sum(CtXtXC.A * SSt.A)
                    #print(SSE)
                    
                    if SSE <= SSE_old * (1 + 1e-9):
                        mualpha = mualpha * 1.2
                        XC = XCt
                        break
                    else:
                        mualpha = mualpha / 2

        if delta != 0:
            C = C * np.diag(alphaC)

        return C, SSE, muC, mualpha, CtXtXC, XC

    N, M = X.shape
    

    if I is None:
        I = range(M)
    if U is None:
        U = range(M)
        
    #print(X[:,U])

    SST = np.sum(X[:, U] * X[:, U])
    #print(SST)
    
    #print(X[:,I])

    #Initialize C
    try:
       i = furthest_sum(X[:, I], noc, [int(np.ceil(len(I) * np.random.rand()))])
    except IndexError:
        class InitializationException(Exception): pass
        raise InitializationException("Initialization does not converge. Too few examples in dataset.")

    #i = [1,2,0]
        
    j = range(noc)
    C = csr_matrix((np.ones(len(i)), (i, j)), shape=(len(I), noc)).todense()
    #print(C)
    
    XC = np.dot(X[:, I], C)
    #print(XC)

    muS, muC, mualpha = 1, 1, 1

    # Initialise S
    XCtX = np.dot(XC.T, X[:, U])
    #print(XCtX)
    CtXtXC = np.dot(XC.T, XC)
    #print(CtXtXC)
    S = -np.log(np.random.random((noc, len(U))))
    #S = -np.log((np.ones((3,48))-0.5))
    #print(S)
    S = S / np.dot(np.ones((noc, 1)), np.mat(np.sum(S, axis=0)))
    #print(S)
    SSt = np.dot(S, S.T)
    SSE = SST - 2 * np.sum(XCtX.A * S.A) + np.sum(CtXtXC.A * SSt.A)
    #print(SSE)
    S, SSE, muS, SSt = S_update(S, XCtX, CtXtXC, muS, SST, SSE, 25)
    #print(S, SSE, muS, SSt)

    # Set PCHA parameters
    iter_ = 0
    dSSE = np.inf
    t1 = dt.now()
    varexpl = (SST - SSE) / SST
    #print(varexpl)

    if verbose:
        print('\nPrincipal Convex Hull Analysis / Archetypal Analysis')
        print('A ' + str(noc) + ' component model will be fitted')
        print('To stop algorithm press control C\n')

    dheader = '%10s | %10s | %10s | %10s | %10s | %10s | %10s | %10s' % ('Iteration', 'Expl. var.', 'Cost func.', 'Delta SSEf.', 'muC', 'mualpha', 'muS', ' Time(s)   ')
    dline = '-----------+------------+------------+-------------+------------+------------+------------+------------+'

    while np.abs(dSSE) >= conv_crit * np.abs(SSE) and iter_ < maxiter and varexpl < 0.9999:
        if verbose and iter_ % 100 == 0:
            print(dline)
            print(dheader)
            print(dline)
        told = t1
        iter_ += 1
        SSE_old = SSE

        # C (and alpha) update
        XSt = np.dot(X[:, U], S.T)
        #print(XSt)
        C, SSE, muC, mualpha, CtXtXC, XC = C_update(
            X[:, I], XSt, XC, SSt, C, delta, muC, mualpha, SST, SSE, 10
        )
        #print(C,SSE,muC,mualpha,CtXtXC,XC)

        # S update
        XCtX = np.dot(XC.T, X[:, U])
        #print(XCtX)
        S, SSE, muS, SSt = S_update(
            S, XCtX, CtXtXC, muS, SST, SSE, 10
        )
        #print(S,SSE,muS,SSt)

        # Evaluate and display iteration
        dSSE = SSE_old - SSE
        #print(dSSE)
        t1 = dt.now()
        if iter_ % 1 == 0:
            time.sleep(0.000001)
            varexpl = (SST - SSE) / SST
            if verbose:
                print('%10.0f | %10.4f | %10.4e | %10.4e | %10.4e | %10.4e | %10.4e | %10.4f \n' % (iter_, varexpl, SSE, dSSE/np.abs(SSE), muC, mualpha, muS, (t1-told).seconds))

    # Display final iteration
    varexpl = (SST - SSE) / SST
    #print(varexpl)
    if verbose:
        print(dline)
        print(dline)
        print('%10.0f | %10.4f | %10.4e | %10.4e | %10.4e | %10.4e | %10.4e | %10.4f \n' % (iter_, varexpl, SSE, dSSE/np.abs(SSE), muC, mualpha, muS, (t1-told).seconds))

    #print(S)
    #print(C)
    #print(XC)
        
    # Sort components according to importance
    ind, vals = zip(
        *sorted(enumerate(np.sum(S, axis=1)), key=lambda x: x[0], reverse=1)
    )
    S = S[ind, :]
    C = C[:, ind]
    XC = XC[:, ind]

    return XC, S, C, SSE, varexpl

XC2, S2, C2, SSE2, varexpl2 = PCHAm(dec6_wo_st, noc=3, I = None, delta=0)

In [ ]:
S_matrix_pd = pd.DataFrame(S2.T)

In [ ]:
dec6_state = dec6_state.reset_index()
dec6_state = dec6_state['state']
dec6_state
S_pd = pd.concat([dec6_state, S_matrix_pd], axis=1)
S_pd
#S_pd = S_pd.set_index('state')

In [ ]:
S_pd.to_csv('/Users/markomiholjcic/Documents/GitHub/ArchetypalAnalysis/data/SdataFrame.csv', index = False)

In [ ]:
dec6_variables = dec6_variables.rename(columns={0: "var"})
dec6_variables

In [ ]:
XC2_pd = pd.DataFrame(XC2)
XC2_pd = pd.concat([dec6_variables,XC2_pd], axis = 1)
#XC2_pd = XC2_pd.set_index("var")
XC2_pd

In [ ]:
XC2_pd.to_csv('/Users/markomiholjcic/Documents/GitHub/ArchetypalAnalysis/data/XC.csv', index = False)

In [ ]:
XC, S, C, SSE, varexpl = PCHA(dec6_wo_st, noc=3, I = None, delta=0)

In [ ]:
S[:,1]

In [ ]:
ans = 0

a = XC[:,0] - XC2[:,1]
ans = ans + a

ans2 = 0

a = XC[:,1] - XC2[:,2]
ans2 = ans2 + a

ans3 = 0

a = XC[:,2] - XC2[:,0]
ans = ans3 + a
    
print(ans, ans2, ans3)

In [ ]:
ans = 0

for i in range(len(S)):
    a = S[i] - S2[i]
    ans = ans + a
    
print(ans)

In [ ]:
ans = 0

for i in range(len(C)):
    a = C[i] - C2[i]
    ans = ans + a
    
print(ans)